In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import tensorflow as tf
import strawberryfields as sf
from strawberryfields.ops import *
import pickle
import random as rd
import warnings
from datetime import datetime
import uncertainty_toolbox as uct
from itertools import product
from tqdm import tqdm

from dataloader import get_data, quartic_data
from plotting import plot_predictions, plot_training_results, plot_predictions_new, plot_eval_metrics
from baseline import train_mlp_baseline, predict_mlp_baseline, train_polynomial_baseline, predict_polynomial_baseline
from uq import selective_prediction, compute_eval_metrics
from model import train_memristor, predict_memristor, build_circuit
from utils import format_metrics, format_hyperparameters
from main import hyperparameter_optimization

tf.get_logger().setLevel('ERROR')
warnings.filterwarnings("ignore")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
rd.seed(42)

In [ ]:
class Config:

    
    LOG_NAME = f"logs_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    LOG_FILE_NAME = f"reports/logs/experiment_{LOG_NAME}/log.txt"
    LOG_PATH = f"reports/logs/experiment_{LOG_NAME}/"
    
    # Hyperparameter Optimization
    HYPERPARAMETER_OPTIMIZATION = False
    HYPER_STEPS_RANGE = [10]
    HYPER_LEARNING_RATE_RANGE = [0.1]
    HYPER_MEMORY_DEPTH_RANGE = [4, 6]
    HYPER_CUTOFF_DIM_RANGE = [4, 6]

    # Model Comparison
    MODEL_COMPARISON = False
    COMP_N_SAMPLES = [2]
    COMP_MLP_ARCH = [[32], [64, 64], [128, 64, 64]]

    #TODO: Check for which params we have the same loss
    MLP_HIDDEN_LAYERS = [64, 64]
    MLP_EPOCHS = 100
    MLP_LEARNING_RATE = 0.01

    POLYNOMIAL_DEGREE = 3

    # Selective Prediction
    SELECTIVE_PREDICTION_THRESHOLD = 0.8

    # QNN Hyperparameters
    MEMORY_DEPTH = 2
    CUTOFF_DIM = 2
    TRAINING_STEPS = 10
    TRAINING_LEARNING_RATE = 0.2

    PREDICT_STOCHASTIC = True
    PREDICT_SAMPLES = 20
    PREDICT_VARIANCE = 0.1

    GET_DATA_N_DATA = 200
    GET_DATA_SIGMA_NOISE_1 = 0.1
    GET_DATA_DATAFUNCTION = quartic_data

    PARAM_ID = f"qnn_hp_s{TRAINING_STEPS}_lr{TRAINING_LEARNING_RATE}_md{MEMORY_DEPTH}_cd{CUTOFF_DIM}"


In [ ]:
# Create directory called experiment_CONFIG.LOG_NAME in reports/logs
os.makedirs(f"reports/logs/experiment_{Config.LOG_NAME}", exist_ok=False)

with open(Config.LOG_FILE_NAME, "a") as f:
    f.write("=" * 80 + "\n")
    f.write(f"Experiment_{Config.LOG_NAME}\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("=" * 80 + "\n\n")


X_train, y_train, X_test, y_test, _ = get_data(n_data=Config.GET_DATA_N_DATA, 
                                                sigma_noise_1=Config.GET_DATA_SIGMA_NOISE_1, 
                                                datafunction=Config.GET_DATA_DATAFUNCTION
                                                )

In [ ]:
# Train model
res_mem, phase1, phase3, memristor_weight = train_memristor(X_train, 
                                                            y_train, 
                                                            memory_depth=Config.MEMORY_DEPTH, 
                                                            training_steps=Config.TRAINING_STEPS,
                                                            learning_rate=Config.TRAINING_LEARNING_RATE,
                                                            cutoff_dim=Config.CUTOFF_DIM,
                                                            log_filepath=Config.LOG_FILE_NAME,
                                                            log_path=Config.LOG_PATH,
                                                            param_id=Config.PARAM_ID
                                                            )

In [ ]:
# Different selective prediction thresholds

PREDICT_STOCHASTIC = True
# PREDICT_SAMPLES = [30, 50, 100]
PREDICT_VARIANCE = 0.1

GET_DATA_N_DATA = 200
GET_DATA_SIGMA_NOISE_1 = 0.1
GET_DATA_DATAFUNCTION = quartic_data

# Selective Prediction
SELECTIVE_PREDICTION_THRESHOLD = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

rmse_values_dic = {str(PREDICT_SAMPLES[i]) : [] for i in range(len(PREDICT_SAMPLES))}

for index, samples in enumerate(sample_storage):
    for threshold in SELECTIVE_PREDICTION_THRESHOLD:
        

        PARAM_ID = f"qnn_stoch{PREDICT_STOCHASTIC}_psamples{PREDICT_SAMPLES}_pvar{PREDICT_VARIANCE}_ndata{GET_DATA_N_DATA}_snoise{GET_DATA_SIGMA_NOISE_1}_threshold{threshold}"

        with open(Config.LOG_FILE_NAME, "a") as f:
                f.write("=" * 80 + "\n")
                f.write(f"Experiment_{PARAM_ID}\n")
                f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write("=" * 80 + "\n\n")

        if PREDICT_STOCHASTIC:
            # Apply selective prediction
            sel_predictions, sel_targets, sel_uncertainty, remaining_fraction = selective_prediction(samples[0], 
                                                                                                    samples[1], 
                                                                                                    samples[2], 
                                                                                                    threshold=threshold
                                                                                                    )
            
            plot_predictions_new(X_test, y_test, predictions, predictive_uncertainty, Config.LOG_PATH+f"prediction_results_{PARAM_ID}.png")

            # Compute evaluation metrics for selective predictions
            sel_metrics, sel_metric_categories = compute_eval_metrics(sel_predictions, 
                                                                    sel_targets, 
                                                                    sel_uncertainty,
                                                                    Config.LOG_FILE_NAME,
                                                                    PARAM_ID,
                                                                    name="Selective Prediction"
                                                                    )
            
            rmse_values_dic[str(PREDICT_SAMPLES[index])].append(sel_metrics['accuracy']["rmse"])

            # Save results to log file
            with open(Config.LOG_FILE_NAME, "a") as f:
                f.write(f"Selective Prediction Fraction: {remaining_fraction}\n")
                f.write("\n\n")

            # Plotting the results
            plot_predictions(
                X_train.numpy(), y_train.numpy(), X_test.numpy(), y_test.numpy(),
                predictions, pred_std=predictive_uncertainty, epistemic=predictive_uncertainty,
                aleatoric=None, title="Memristor Model Predictions vs Targets", save_path=Config.LOG_PATH+f"prediction_uncertainty_{PARAM_ID}.png"
            )